# Programming Assignment 4: Information Retrieval

In this assignment you will be improving upon a rather poorly-made information retrieval system. You will build an inverted index to quickly retrieve documents that match queries and then make it even better by using term-frequency inverse-document-frequency weighting and cosine similarity to compare queries to your data set. Your IR system will be evaluated for accuracy on the correct documents retrieved for different queries and the correctly computed tf-idf values and cosine similarities.

## Data

You will be using your IR system to find relevant documents among a collection of sixty short stories by Rider Haggard. The training data is located in the data/ directory under the subdirectory RiderHaggard/. Within this directory you will see yet another directory raw/. This contains the raw text files of the sixty short stories. The data/ directory also contains the files dev_queries.txt and dev_solutions.txt. We have provided these to you as a set of development queries and their expected answers to use as you begin implementing your IR system.

## Your Task

Improve upon the given IR system by implementing the following features:

<b>Inverted Positional Index:</b> Implement an inverted index - a mapping from words to the documents in which they occur, as well as the positions in the documents for which they occur.

<b>Boolean Retrieval:</b> Implement a Boolean retrieval system, in which you return the list of documents that contain all words in a query. (Yes, you only need to support conjunctions for this assignment.)

<b>Phrase Query Retrieval:</b> Implement a system that returns the list of documents in which the full phrase appears, (ie. the words of the query appear next to each other, in the specified order). Note that at the time of retrieval, you will not have access to the original documents anymore (the documents would be turned into bag of words), so you'll have to utilize your inverted positional index to complete this part.

<b>TF-IDF:</b> Compute and store the term-frequency inverse-document-frequency value for every word-document co-occurrence:

<b>Cosine Similarity:</b> Implement cosine similarity in order to improve upon the ranked retrieval system, which currently retrieves documents based upon the Jaccard coefficient between the query and each document. Also note that when computing $w_{t, q}$ (i.e. the weight for the word 𝑤 in the query) do not include the idf term. That is, $w_{t, q} = 1 + \log_{10} \text{tf}_{t, q}$.
<b> The reference solution uses ltc.lnn weighting for computing cosine scores. </b>

## Evaluation

Your IR system will be evaluated on a development set of queries as well as a held-out set of queries. The queries are encoded in the file <b>dev_queries.txt</b> and are:

- separation of church and state
- white-robed priests
- ancient underground city
- native african queen
- zulu king

We test your system based on the five parts mentioned above: the inverted index (used both to get word positions and to get postings), boolean retrieval, phrase query retrieval, computing the correct tf-idf values, and implementing cosine similarity using the tf-idf values.

## Getting Started!

We will first start by importing some modules and setting up our IR system class.


In [27]:
import json
import os
import re
import sys
import math

from porter_stemmer import PorterStemmer

In [28]:
"""
DO NOT CHANGE
"""
class IRSystem:
    def __init__(self):
        # For holding the data - initialized in read_data()
        self.titles = []
        self.docs = []
        self.vocab = []
        # For the text pre-processing.
        self.alphanum = re.compile('[^a-zA-Z0-9]')
        self.p = PorterStemmer()

    def get_uniq_words(self):
        uniq = set()
        for doc in self.docs:
            for word in doc:
                uniq.add(word)
        return uniq

    def __read_raw_data(self, dirname):
        print("Stemming Documents...")

        titles = []
        docs = []
        os.mkdir('%s/stemmed' % dirname)
        title_pattern = re.compile('(.*) \d+\.txt')

        # make sure we're only getting the files we actually want
        filenames = []
        for filename in os.listdir('%s/raw' % dirname):
            if filename.endswith(".txt") and not filename.startswith("."):
                filenames.append(filename)

        for i, filename in enumerate(filenames):
            title = title_pattern.search(filename).group(1)
            print("    Doc %d of %d: %s" % (i + 1, len(filenames), title))
            titles.append(title)
            contents = []
            f = open('%s/raw/%s' % (dirname, filename), 'r', encoding="utf-8")
            of = open('%s/stemmed/%s.txt' % (dirname, title), 'w',
                      encoding="utf-8")
            for line in f:
                # make sure everything is lower case
                line = line.lower()
                # split on whitespace
                line = [xx.strip() for xx in line.split()]
                # remove non alphanumeric characters
                line = [self.alphanum.sub('', xx) for xx in line]
                # remove any words that are now empty
                line = [xx for xx in line if xx != '']
                # stem words
                line = [self.p.stem(xx) for xx in line]
                # add to the document's conents
                contents.extend(line)
                if len(line) > 0:
                    of.write(" ".join(line))
                    of.write('\n')
            f.close()
            of.close()
            docs.append(contents)
        return titles, docs

    def __read_stemmed_data(self, dirname):
        print("Already stemmed!")
        titles = []
        docs = []

        # make sure we're only getting the files we actually want
        filenames = []
        for filename in os.listdir('%s/stemmed' % dirname):
            if filename.endswith(".txt") and not filename.startswith("."):
                filenames.append(filename)

        if len(filenames) != 60:
            msg = "There are not 60 documents in ../data/RiderHaggard/stemmed/\n"
            msg += "Remove ../data/RiderHaggard/stemmed/ directory and re-run."
            raise Exception(msg)

        for i, filename in enumerate(filenames):
            title = filename.split('.')[0]
            titles.append(title)
            contents = []
            f = open('%s/stemmed/%s' % (dirname, filename), 'r',
                     encoding="utf-8")
            for line in f:
                # split on whitespace
                line = [xx.strip() for xx in line.split()]
                # add to the document's conents
                contents.extend(line)
            f.close()
            docs.append(contents)

        return titles, docs

    def read_data(self, dirname):
        """
        Given the location of the 'data' directory, reads in the documents to
        be indexed.
        """
        # NOTE: We cache stemmed documents for speed
        #       (i.e. write to files in new 'stemmed/' dir).

        print("Reading in documents...")
        # dict mapping file names to list of "words" (tokens)
        filenames = os.listdir(dirname)
        subdirs = os.listdir(dirname)
        if 'stemmed' in subdirs:
            titles, docs = self.__read_stemmed_data(dirname)
        else:
            titles, docs = self.__read_raw_data(dirname)

        # Sort document alphabetically by title to ensure we have the proper
        # document indices when referring to them.
        ordering = [idx for idx, title in sorted(enumerate(titles),
                                                 key=lambda xx: xx[1])]

        self.titles = []
        self.docs = []
        numdocs = len(docs)
        for d in range(numdocs):
            self.titles.append(titles[ordering[d]])
            self.docs.append(docs[ordering[d]])

        # Get the vocabulary.
        self.vocab = [xx for xx in self.get_uniq_words()]

You will first build the inverted positional index (data structure that keeps track of the documents in which a particular word is contained, and the positions of that word in the document). The documents will have already been read in at this point. The following instance variables in the class are included in the starter code for you to use to build your inverted positional index: titles (a list of strings), docs (a list of lists of strings), and vocab (a list of strings).

In [29]:
class IRSystem(IRSystem):
    def index(self):
        """
        Build an index of the documents.
        """
        print("Indexing...")
        # ------------------------------------------------------------------
        # TODO: Create an inverted, positional index.
        #       Granted this may not be a linked list as in a proper
        #       implementation.
        #       This index should allow easy access to both 
        #       1) the documents in which a particular word is contained, and 
        #       2) for every document, the positions of that word in the document 
        #       Some helpful instance variables:
        #         * self.docs = List of documents
        #         * self.titles = List of titles

        inv_index = {}

        # Generate inverted index here
        for i in range(len(self.docs)):
            curr_doc = self.docs[i] #List of words (the story)

            for posting_idx in range(len(curr_doc)):
                word = curr_doc[posting_idx]
                if word not in inv_index:
                    inv_index[word] = [0, {}]
                doc_dict = inv_index[word][1]
                if self.titles[i] not in doc_dict:
                    doc_dict[self.titles[i]] = []
                    inv_index[word][0] += 1
                (doc_dict[self.titles[i]]).append(posting_idx)


        self.inv_index = inv_index

        # ------------------------------------------------------------------

        # turn self.docs into a map from ID to bag of words
        id_to_bag_of_words = {}
        for d, doc in enumerate(self.docs):
            bag_of_words = set(doc)
            id_to_bag_of_words[d] = bag_of_words
        self.docs = id_to_bag_of_words

Next we will implement get_word_positions. This method returns a list of integers that identifies the positions in the document doc in which the word is found.  This is basically just an API into your inverted index, but you must implement it in order for the index to be evaluated fully.

In [30]:
class IRSystem(IRSystem):
    def get_word_positions(self, word, doc):
        """
        Given a word and a document, use the inverted index to return
        the positions of the specified word in the specified document.
        """
        # ------------------------------------------------------------------
        # TODO: return the list of positions for a word in a document.
        positions = []
        doc_title = self.titles[doc]
        for position in self.inv_index[word][1][doc_title]:
            positions.append(position)

        return positions
        # ------------------------------------------------------------------

We will add another method, get_posting, that returns a list of integers (document IDs) that identifies the documents in which the word is found. This is basically another API into your inverted index, but you must implement it in order to be evaluated fully.

Keep in mind that the document IDs in each postings list to be sorted in order to perform the linear merge for boolean retrieval.

Next, we will implement Boolean retrieval, returning a list of document IDs corresponding to the documents in which all the words in query occur.

Please implement the linear merge algorithm outlined in the videos/book (do not use built-in set intersection functions).

In [31]:
class IRSystem(IRSystem):
    def get_posting(self, word):
        """
        Given a word, this returns the list of document indices (sorted) in
        which the word occurs.
        """
        # ------------------------------------------------------------------
        # TODO: return the list of postings for a word.
        posting = []
        doc_titles= []
        for title in self.inv_index[word][1]:
            doc_titles.append(title)

        for idx in range(len(self.titles)):
            if self.titles[idx] in doc_titles:
                posting.append(idx)

        return posting
        # ------------------------------------------------------------------

    def get_posting_unstemmed(self, word):
        """
        Given a word, this *stems* the word and then calls get_posting on the
        stemmed word to get its postings list. You should *not* need to change
        this function. It is needed for submission.
        """
        word = self.p.stem(word)
        return self.get_posting(word)

In [32]:
class IRSystem(IRSystem):
    def boolean_retrieve(self, query):
        """
        Given a query in the form of a list of *stemmed* words, this returns
        the list of documents in which *all* of those words occur (ie an AND
        query).
        Return an empty list if the query does not return any documents.
        """
        # ------------------------------------------------------------------
        # TODO: Implement Boolean retrieval. You will want to use your
        #       inverted index that you created in index().
        # Right now this just returns all the possible documents!
        min_num_docs = 61 # There are a total of 60 documents
        min_doc_word = ''
        for word in query:
            if self.inv_index[word][0] < min_num_docs:
                min_doc_word = word

        def isOccuring(title):
            for word in query:
                if title not in self.inv_index[word][1]:
                    return False
            return True

        docs_titles = []
        for title in self.inv_index[min_doc_word][1]:
            if isOccuring(title):
                docs_titles.append(title)

        docs = [] # List of docIDs
        for idx in range(len(self.titles)):
            if self.titles[idx] in docs_titles:
                docs.append(idx)

        # ------------------------------------------------------------------

        return sorted(docs)  # sorted doesn't actually matter

Let's continue to phase query retrieval. Our phrase_retrieve method will return a list of document IDs corresponding to the documents in which all the actual query phrase occurs.

In [33]:
class IRSystem(IRSystem):
    def phrase_retrieve(self, query):
        """
        Given a query in the form of an ordered list of *stemmed* words, this 
        returns the list of documents in which *all* of those words occur, and 
        in the specified order. 
        Return an empty list if the query does not return any documents. 
        """
        # ------------------------------------------------------------------
        # TODO: Implement Phrase Query retrieval (ie. return the documents 
        #       that don't just contain the words, but contain them in the 
        #       correct order) You will want to use the inverted index 
        #       that you created in index(), and may also consider using
        #       boolean_retrieve. 
        #       NOTE that you no longer have access to the original documents
        #       in self.docs because it is now a map from doc IDs to set
        #       of unique words in the original document.
        # Right now this just returns all possible documents!

        potential_docIDs = self.boolean_retrieve(query)
        potential_titles = []
        for docID in potential_docIDs:
            potential_titles.append(self.titles[docID])

        def isPhrase(query, title, doc_titles):
            all_postings = []
            for word in query:
                postings_list = self.inv_index[word][1][title].copy()
                all_postings.append(postings_list)
            for posting in all_postings[0]:
                recursive_find(all_postings, posting, 1, doc_titles, title)


        def recursive_find(all_postings, posting, depth, doc_titles, title):
            if depth == len(all_postings):
                doc_titles.append(title)
            else:
                if posting + 1 in all_postings[depth]:
                    recursive_find(all_postings, posting + 1, depth + 1, doc_titles, title)

        doc_titles = []
        for title in potential_titles:
            isPhrase(query, title, doc_titles)


        docs = [] # List of docIDs
        for idx in range(len(self.titles)):
            if self.titles[idx] in doc_titles:
                docs.append(idx)

        # ------------------------------------------------------------------

        return sorted(docs)  # sorted doesn't actually matter

Now we will compute and score the td-idf values. compute_tfidf and stores the tf-idf values for words and documents. For this you will probably want to be aware of the class variable vocab, which holds the list of all unique words, as well as the inverted index you created earlier.

You must also implement get_tfidf to return the tf-idf weight for a particular word and document ID.

In [34]:
class IRSystem(IRSystem):
    def compute_tfidf(self):
        # -------------------------------------------------------------------
        # TODO: Compute and store TF-IDF values for words and documents in self.tfidf.
        #       Recall that you can make use of:
        #         * self.vocab: a list of all distinct (stemmed) words
        #         * self.docs: a list of lists, where the i-th document is
        #                   self.docs[i] => ['word1', 'word2', ..., 'wordN']
        #       NOTE that you probably do *not* want to store a value for every
        #       word-document pair, but rather just for those pairs where a
        #       word actually occurs in the document.
        print("Calculating tf-idf...")
        self.tfidf = {}
        N = len(self.docs)

        for word in self.vocab:
            for title in self.inv_index[word][1]:
                tf = 1 + math.log(len(self.inv_index[word][1][title]), 10)
                idf = math.log(N / (self.inv_index[word][0]), 10)
                self.tfidf[(word, title)] = tf * idf

        # ------------------------------------------------------------------

    def get_tfidf(self, word, document):
        # ------------------------------------------------------------------
        # TODO: Return the tf-idf weigthing for the given word (string) and
        #       document index.
        tfidf = self.tfidf[(word, self.titles[document])]
        # ------------------------------------------------------------------
        return tfidf

    def get_tfidf_unstemmed(self, word, document):
        """
        This function gets the TF-IDF of an *unstemmed* word in a document.
        Stems the word and then calls get_tfidf. You should *not* need to
        change this interface, but it is necessary for submission.
        """
        word = self.p.stem(word)
        return self.get_tfidf(word, document)

Lastly, we will implement rank_retrieve. This function returns a priority queue of the top ranked documents for a given query. Right now it ranks documents according to their Jaccard similarity with the query, but you will replace this method of ranking with a ranking using the <b>cosine similarity</b> between the documents and query.
    
Remember to use ltc.lnn weighting, that is, ltc weighting for the document and lnn weighting for the query! This means that the query vector weights will be $1 + \log_{10} \text{tf}_{t, q}$ with no IDF term or normalization, but we normalize the document vector weights by the length of the document vector (square root of the sum of squares of the tf-idf weights). Finally, the cosine scores are the dot product of the query vector and the document vectors. Refer to this [handout](http://web.stanford.edu/class/cs124/lec/CS124_IR_Handout.pdf) or lecture slides for a more detailed explanation.
    
When we say normalize by "document length" or "length of document", we mean the length of the document vector, NOT the number of words in the actual text document. So, when you’re computing cosine similarity between the query and document, the document vector is the tf.idf document weights for all terms in the vocabulary. The document length would be the L2 norm of the document vector.

In [35]:
class IRSystem(IRSystem):
    def rank_retrieve(self, query):
        """
        Given a query (a list of words), return a rank-ordered list of
        documents (by ID) and score for the query.
        """
        scores = [0.0 for xx in range(len(self.titles))]
        # ------------------------------------------------------------------
        # TODO: Implement cosine similarity between a document and a list of
        #       query words.

        # Right now, this code simply gets the score by taking the Jaccard
        # similarity between the query and every document.
        words_in_query = set()
        for word in query:
            words_in_query.add(word)

        query_word_dict = {}
        for word in query:
            if word not in query_word_dict:
                query_word_dict[word] = 0
            query_word_dict[word] += 1

        ltc_map = {}
        for word in words_in_query:
            ltc_map[word] = 1 + math.log(query_word_dict[word], 10)


        for d in range(len(self.docs)):
            doc_title = self.titles[d]
            sum_weight = 0
            words_in_doc = self.docs[d]
            for word in words_in_doc:
                sum_weight += (self.tfidf[(word, doc_title)])**2
            length_doc = math.sqrt(sum_weight)

            score = 0
            for word in words_in_query.intersection(words_in_doc):
                score += ltc_map[word] * (self.tfidf[(word, doc_title)] / length_doc)

            scores[d] = score
        # ------------------------------------------------------------------

        ranking = [idx for idx, sim in sorted(enumerate(scores),
                                              key=lambda xx: xx[1],
                                              reverse=True)]
        results = []
        for i in range(10):
            results.append((ranking[i], scores[ranking[i]]))
        return results

Let's also add a few methods that given a string, will process and then return the list of matching documents for the different methods you have implemented. You do not need to add any additional code here.

In [36]:
"""
DO NOT CHANGE
"""
class IRSystem(IRSystem):
    def process_query(self, query_str):
        """
        Given a query string, process it and return the list of lowercase,
        alphanumeric, stemmed words in the string.
        """
        # make sure everything is lower case
        query = query_str.lower()
        # split on whitespace
        query = query.split()
        # remove non alphanumeric characters
        query = [self.alphanum.sub('', xx) for xx in query]
        # stem words
        query = [self.p.stem(xx) for xx in query]
        return query

    def query_retrieve(self, query_str):
        """
        Given a string, process and then return the list of matching documents
        found by boolean_retrieve().
        """
        query = self.process_query(query_str)
        return self.boolean_retrieve(query)

    def phrase_query_retrieve(self, query_str):
        """
        Given a string, process and then return the list of matching documents
        found by phrase_retrieve().
        """
        query = self.process_query(query_str)
        return self.phrase_retrieve(query)

    def query_rank(self, query_str):
        """
        Given a string, process and then return the list of the top matching
        documents, rank-ordered.
        """
        query = self.process_query(query_str)
        return self.rank_retrieve(query)

## Running the code
You can use the run_query function to test your code on a specific query. 

Note that the first time to run the run_query function, it will create a directory named stemmed/ in ../data/RiderHaggard/. This is meant to be a simple cache for the raw text documents. Later runs will be much faster after the first run. However, this means that if something happens during this first run and it does not get through processing all the documents, you may be left with an incomplete set of documents in ../data/RiderHaggard/stemmed/. If this happens, simply remove the stemmed/ directory and re-run!

In [37]:
"""
DO NOT CHANGE
"""
def run_tests(irsys):
    print("===== Running tests =====")

    ff = open('./data/dev_queries.txt')
    questions = [xx.strip() for xx in ff.readlines()]
    ff.close()
    ff = open('./data/dev_solutions.txt')
    solutions = [xx.strip() for xx in ff.readlines()]
    ff.close()

    total_score = 0

    epsilon = 1e-4
    for part in range(6):
        points = 0
        num_correct = 0
        num_total = 0

        prob = questions[part]
        soln = json.loads(solutions[part])

        if part == 0:  # inverted index test
            print("Inverted Index Test")
            queries = prob.split("; ")
            queries = [xx.split(", ") for xx in queries]
            queries = [(xx[0], int(xx[1])) for xx in queries]
            for i, (word, doc) in enumerate(queries):
                num_total += 1
                guess = irsys.get_word_positions(word, doc)
                if sorted(guess) == soln[i]:
                    num_correct += 1

        if part == 1:  # get postings test
            print("Get Postings Test")
            words = prob.split(", ")
            for i, word in enumerate(words):
                num_total += 1
                posting = irsys.get_posting_unstemmed(word)
                if posting == soln[i]:
                    num_correct += 1

        elif part == 2:  # boolean retrieval test
            print("Boolean Retrieval Test")
            queries = prob.split(", ")
            for i, query in enumerate(queries):
                num_total += 1
                guess = irsys.query_retrieve(query)
                if set(guess) == set(soln[i]):
                    num_correct += 1

        elif part == 3:  # phrase query test
            print("Phrase Query Retrieval")
            queries = prob.split(", ")
            for i, query in enumerate(queries):
                num_total += 1
                guess = irsys.phrase_query_retrieve(query)
                if set(guess) == set(soln[i]):
                    num_correct += 1

        elif part == 4:  # tfidf test
            print("TF-IDF Test")
            queries = prob.split("; ")
            queries = [xx.split(", ") for xx in queries]
            queries = [(xx[0], int(xx[1])) for xx in queries]
            for i, (word, doc) in enumerate(queries):
                num_total += 1
                guess = irsys.get_tfidf_unstemmed(word, doc)
                if guess >= float(soln[i]) - epsilon and guess <= float(soln[i]) + epsilon:
                    num_correct += 1

        elif part == 5:  # cosine similarity test
            print("Cosine Similarity Test")
            queries = prob.split(", ")
            for i, query in enumerate(queries):
                num_total += 1
                ranked = irsys.query_rank(query)
                top_rank = ranked[0]
                if top_rank[0] == soln[i][0]:
                    if top_rank[1] >= float(soln[i][1]) - epsilon and top_rank[1] <= float(soln[i][1]) + epsilon:
                        num_correct += 1

        feedback = "%d/%d Correct. Accuracy: %f" % (num_correct, num_total, float(num_correct) / num_total)

        if part == 1:
            if num_correct == num_total:
                points = 2
            elif num_correct >= 0.5 * num_total:
                points = 1
            else:
                points = 0
        elif part == 2:
            if num_correct == num_total:
                points = 1
            else:
                points = 0
        else:
            if num_correct == num_total:
                points = 3
            elif num_correct > 0.75 * num_total:
                points = 2
            elif num_correct > 0:
                points = 1
            else:
                points = 0
        total_score += points
        print("    Score: %d Feedback: %s" % (points, feedback))

    return total_score

In [38]:
"""
DO NOT CHANGE
"""
## Run this cell to run the tests
irsys = IRSystem()
irsys.read_data('./data/RiderHaggard')
irsys.index()
irsys.compute_tfidf()

import math

total_score = 15
final_score = math.ceil(100 * run_tests(irsys) / total_score)
print(f"\nFinal Score: {final_score} out of 100")

Reading in documents...
Already stemmed!
Indexing...
Calculating tf-idf...
===== Running tests =====
Inverted Index Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000
Get Postings Test
    Score: 2 Feedback: 5/5 Correct. Accuracy: 1.000000
Boolean Retrieval Test
    Score: 1 Feedback: 5/5 Correct. Accuracy: 1.000000
Phrase Query Retrieval
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000
TF-IDF Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000
Cosine Similarity Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000

Final Score: 100 out of 100


In [39]:
def run_query(query):
    irsys = IRSystem()
    irsys.read_data('./data/RiderHaggard')
    irsys.index()
    irsys.compute_tfidf()

    print("Best matching documents to '%s':" % query)
    results = irsys.query_rank(query)
    for docId, score in results:
        print("%s: %e" % (irsys.titles[docId], score))


## Example query run where "My very own query" is your query.
## TODO test your own query
run_query("My very own query")

Reading in documents...
Already stemmed!
Indexing...
Calculating tf-idf...
Best matching documents to 'My very own query':
Elissa: 1.321682e-02
Heu-Heu (1924): 1.152362e-02
The Ghost Kings: 1.137714e-02
Stories by English Authors Africa (Selected by Scribners): 1.124628e-02
Love Eternal: 1.076495e-02
Lysbeth, a Tale of the Dutch: 9.444311e-03
The Lady of Blossholme: 9.299776e-03
Pearl-Maiden: 9.241841e-03
Stella Fregelius: 8.908197e-03
Moon of Israel: 8.860210e-03


## Information Retrieval Systems and Search Engines

Safiya Umoja Noble’s <a href="https://nyupress.org/9781479837243/algorithms-of-oppression/">Algorithms of Oppression</a> (2018) provides insight into how search engines and information retrieval algorithms can exhibit substantial racist and sexist biases. Noble demonstrates how prejudice against black women is embedded into search engine ranked results. These biases are apparent in both Google search’s autosuggestions and the first page of Google results. In this assignment, we have explored numerous features that are built into information retrieval systems, like Google Search. 

How could the algorithms you built in this assignment contribute to enforcing real-world biases? 

How can we reduce data discrimination and algorithmic bias that perpetuate gender and racial inequalities in search results and IR system?

Please provide a short response to each of these questions (1-2 paragraphs per question).

In [40]:
class IRSystem(IRSystem):
    def algorithmic_bias_in_IR_systems(self):
        # TODO: Place your response into the response string below
        response = "Q1: Our algorithm's training data is based off of the work of H.Rider Haggard. His novels were written in the 19th and early 20th century, and revolved around adventures in 'exotic locations'. His works have been criticized for their depiciting non-Europeans in a racist and misogynistic manner. In turn, our algorithm learns this behavior and is more likely to correlate racial minorities and women with negative traits. Q2: We can reduce data descrimination and algorithmic bias by carefully identifying the source of bias in the data set and removing it. We should also use a wider set of data that covers multiple persepctives and the opinions of marginalized communities."
        return response